In [158]:
import pandas as pd
import networkx as nx
import matplotlib
import matplotlib.pyplot as plt

from oemof.solph import (Sink, Source, Transformer, Bus, Flow, Model,
                         EnergySystem, Investment, NonConvex)
import oemof.outputlib as outputlib
import oemof.solph as solph
import oemof.graph as graph
from oemof.tools import economics

import pyomo.environ as po
import numpy as np

solver = 'cbc'

%matplotlib inline
matplotlib.rcParams['figure.figsize'] = [8.0, 6.0]

In [2]:
color_dict ={
             'coal': '#755d5d',
             'gas': '#c76c56',
             'oil': '#494a19',
             'lignite': '#56201d',
             'wind': '#4ca7c3',
             'pv': '#ffde32',
             'excess_el': '#9a9da1',
             'pp_coal': '#755d5d',
             'pp_gas': '#c76c56',
             'pp_chp': '#eeac7e',
             'b_heat_source': '#cd3333',
             'heat_source': '#cd3333',
             'heat_pump': '#42c77a',
             'electricity': '#0079ff',
             'demand_el': '#0079ff',
             'shortage_el': '#ff2626',
             'excess_el': '#ff2626',
             'biomass': '#01b42e',
             'pp_biomass': '#01b42e'}

In [3]:
def initialize_basic_energysystem():
    # initialize and provide data
    datetimeindex = pd.date_range('1/1/2016', periods=24*10, freq='H')
    filename = 'input_data.csv'
    data = pd.read_csv(filename, sep=",")
    energysystem = EnergySystem(timeindex=datetimeindex)

    # buses
    bcoal = Bus(label='coal', balanced=False)
    bgas = Bus(label='gas', balanced=False)
    bel = Bus(label='electricity')
    energysystem.add(bcoal, bgas, bel)

    # sources
    energysystem.add(Source(label='wind', outputs={bel: Flow(
        actual_value=data['wind'], nominal_value=66.3, fixed=True)}))

    energysystem.add(Source(label='pv', outputs={bel: Flow(
        actual_value=data['pv'], nominal_value=65.3, fixed=True)}))

    # excess and shortage to avoid infeasibilies
    energysystem.add(Sink(label='excess_el', inputs={bel: Flow()}))
    energysystem.add(Source(label='shortage_el',
                         outputs={bel: Flow(variable_costs=200)}))

    # demands (electricity/heat)
    energysystem.add(Sink(label='demand_el', inputs={bel: Flow(
        nominal_value=85, actual_value=data['demand_el'], fixed=True)}))
    
    return bcoal, bgas, bel, energysystem

In [4]:
bcoal, bgas, bel, energysystem = initialize_basic_energysystem()

# power plants
energysystem.add(Transformer(
    label='pp_coal',
    inputs={bcoal: Flow()},
    outputs={bel: Flow(nominal_value=20.2, variable_costs=25)},
    conversion_factors={bel: 0.39}))

pp_gas = Transformer(
    label='pp_gas',
    inputs={bgas: Flow()},
    outputs={bel: Flow(nominal_value=41, variable_costs=40)},
    conversion_factors={bel: 0.50})
energysystem.add(pp_gas)

In [180]:
bcoal, bgas, bel, energysystem = initialize_basic_energysystem()

# power plants
energysystem.add(Transformer(
    label='pp_coal',
    inputs={bcoal: Flow()},
    outputs={bel: Flow(nominal_value=20.2, variable_costs=25)},
    conversion_factors={bel: 0.39}))

pp_gas = Transformer(
    label='pp_gas',
    inputs={bgas: Flow()},
    outputs={bel: Flow(nominal_value=41, variable_costs=40)},
    conversion_factors={bel: 0.50})
energysystem.add(pp_gas)

# create the model
optimization_model = Model(energysystem)

# add specific emission values to flow objects if source is a commodity bus
for s, t in optimization_model.flows.keys():
    if s is bgas:
        optimization_model.flows[s, t].emission_factor = 0.27  # t/MWh
    if s is bcoal:
        optimization_model.flows[s, t].emission_factor = 0.39  # t/MWh
emission_limit = 60e1

# add the outflow share
optimization_model.flows[(bgas, pp_gas)].outflow_share = [1] * 240

# Add a new pyomo Block
myblock = po.Block()

# create a pyomo set with the flows (i.e. list of tuples),
# there will of course be only one flow inside this set, the one we used to
# add outflow_share
myblock.MYFLOWS = [k for (k, v) in optimization_model.flows.items()
                                     if hasattr(v, 'outflow_share')]


# pyomo does not need a po.Set, we can use a simple list as well
myblock.COMMODITYFLOWS = [k for (k, v) in optimization_model.flows.items()
                          if hasattr(v, 'emission_factor')]

# add the sub-model to the oemof Model instance
optimization_model.add_component('MyBlock', myblock)

def _inflow_share_rule(model, s, e, time):
    """pyomo rule definition: Here we can use all objects from the block or
    the om object, in this case we don't need anything from the block
    except the newly defined set MYFLOWS.
    """
    expr = (optimization_model.flow[s, e, time] >= optimization_model.flows[s, e].outflow_share[time] *
            sum(optimization_model.flow[i, o, time] for (i, o) in optimization_model.FLOWS if o == e))
    return expr

# Add inflow share constraint
myblock.inflow_share = po.Constraint(myblock.MYFLOWS, optimization_model.TIMESTEPS,
                                     rule=_inflow_share_rule)
# add emission constraint
myblock.emission_constr = po.Constraint(expr=(
        sum(optimization_model.flow[i, o, t]
            for (i, o) in myblock.COMMODITYFLOWS
            for t in optimization_model.TIMESTEPS) <= emission_limit))

# add piecewise linear constraint
# def f(model,x):
#     return abs(x-1)+1.0

# optimization_model.X = po.Var(bounds=(-5,5))
# optimization_model.Z = po.Var()
# optimization_model.con = po.Piecewise(optimization_model.Z, optimization_model.X, # range and domain variables
#                       pw_pts=[-5,1,5] ,
#                       pw_constr_type='LB',
#                       f_rule=f)


# add piecewise linear constraint

In [181]:
# Define the function
# Just like in Pyomo constraint rules, a Pyomo model object
# must be the first argument for the function rule
def f(optimization_model,t1,t2,x):
    return 0.1*x - np.cos(5.0*x)

# Note we can use an arbitrary number of index sets of 
# arbitrary dimension as the first arguments to the
# Piecewise component.
# optimization_model.INDEX1 = po.Set(dimen=2, initialize=[(0,1),(8,3)])
optimization_model.X = po.Var(optimization_model.TIMESTEPS, bounds=(-2,2))
optimization_model.Z = po.Var(optimization_model.TIMESTEPS)

# For indexed variables, pw_pts must be a
# python dictionary with keys the same as the variable index
PW_PTS = {}

# Increase n to see the solution approach:
# Z[i]=1.19, X[i]=1.89, obj=7.13
n = 3
# Using piecewise representations with a logarithmic number of
# binary variables ('LOG', 'DLOG') requires that pw_pts lists
# must have 2^n + 1 breakpoints.
num_points = 1 + 2**n
step = (2.0 - (-2.0))/(num_points-1)
for idx in optimization_model.X.index_set():
    PW_PTS[idx] = [-2.0 + i*step for i in range(num_points)]   # [-2.0, ..., 2.0]

optimization_model.linearized_constraint = po.Piecewise(optimization_model.TIMESTEPS,        # indexing sets
                                        optimization_model.Z,optimization_model.X,     # range and domain variables
                                        pw_pts=PW_PTS,
                                        pw_constr_type='EQ',
                                        pw_repn='LOG',
                                        f_rule=f,
                                        force_pw=True)

ERROR:pyomo.core:Constructing component 'linearized_constraint' from data=None failed:
TypeError: f() missing 1 required positional argument: 'x'


ERROR: Constructing component 'linearized_constraint' from data=None failed:
    TypeError: f() missing 1 required positional argument: 'x'


TypeError: f() missing 1 required positional argument: 'x'

In [144]:
nextblock = po.Block()
nextblock.NEXTSET = []
lb = [-5 for i in optimization_model.TIMESTEPS]
ub = [5 for i in optimization_model.TIMESTEPS]
def bounds(optimization_model, i):
    return (lb[i], ub[i])
nextblock.X = po.Var(optimization_model.TIMESTEPS, bounds=bounds)# obounds=(-5,5))
nextblock.Z = po.Var()
optimization_model.add_component('NextBlock', nextblock)

def next_rule(model, x):
    return abs(x-1)+1.0

nextblock.const = po.Piecewise(nextblock.Z, nextblock.X, # range and domain variables
                               pw_pts=[-5,1,5] ,
                               pw_constr_type='LB',
                               f_rule=next_rule)

ERROR:pyomo.core:Constructing component 'NextBlock.const' from data=None failed:
AttributeError: 'IndexedVar' object has no attribute 'lb'


ERROR: Constructing component 'NextBlock.const' from data=None failed:
    AttributeError: 'IndexedVar' object has no attribute 'lb'


AttributeError: 'IndexedVar' object has no attribute 'lb'

In [161]:
# solve problem
optimization_model.solve(solver=solver,
                         solve_kwargs={'tee': False, 'keepfiles': False})

{'Problem': [{'Name': 'unknown', 'Lower bound': 2440575.0624654, 'Upper bound': 2440575.0624654, 'Number of objectives': 1, 'Number of constraints': 980, 'Number of variables': 1469, 'Number of nonzeros': 2509, 'Sense': 'minimize'}], 'Solver': [{'Status': 'ok', 'User time': -1.0, 'Termination condition': 'optimal', 'Error rc': 0, 'Time': 0.031392574310302734}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [162]:
from pyomo.core.base.var import Var
block_vars = []
for bv in optimization_model.component_data_objects(Var):
    block_vars.append(bv.parent_component())
block_vars

 ...]

In [163]:
block_vars = set(block_vars)

In [164]:
results = outputlib.processing.results(optimization_model)

results_el = outputlib.views.node(results, 'electricity')

el_sequences = results_el['sequences']
el_prod = el_sequences[[(('wind', 'electricity'), 'flow'),
                        (('pv', 'electricity'), 'flow'),
                        (('pp_gas', 'electricity'), 'flow'),
                        (('shortage_el', 'electricity'), 'flow')]]

fig, ax = plt.subplots(figsize=(14, 6))
for key in el_sequences.keys():
    color_dict[key] = color_dict[key[0][0]]
c=[color_dict.get(x, '#333333') for x in el_prod.columns]
el_prod.plot.area(ax=ax, color=c)
el_sequences[(('electricity', 'demand_el'), 'flow')].plot(ax=ax, linewidth=3, c='k')
legend = ax.legend(loc='center left', bbox_to_anchor=(1.0, 0.5)) # legend outside of plot

TypeError: 'NoneType' object is not iterable